In [ ]:
# !pip install selenium

In [4]:
from selenium import webdriver
import time
import datetime

# Configura el controlador de Chrome
driver = webdriver.Chrome()

driver.get('https://www.mtgprice.com/sets/Ravnica_Allegiance/Breeding_Pool')

prices=driver.execute_script('return results')

In [11]:
prices[4]

{'color': 'rgb(50, 192, 240)',
 'data': [[1569125358276, 12.133],
  [1569245915662, 11.5815],
  [1569328849786, 12.0835],
  [1569508779622, 12.0802],
  [1569597038099, 12.0318],
  [1569774314067, 12.0285],
  [1569811316326, 12.0285],
  [1569911663627, 10.889],
  [1569936781764, 10.889],
  [1569995321973, 10.898],
  [1570085635146, 11.90825],
  [1570200886417, 11.93659],
  [1570341825204, 11.96711],
  [1570513084476, 11.98237],
  [1570629975128, 12.0846],
  [1570691420494, 12.0791],
  [1570758898328, 12.133],
  [1570779475767, 12.133],
  [1570865822644, 13.240557],
  [1570952242785, 13.240557],
  [1571557155787, 13.93],
  [1571643507094, 13.93],
  [1571902680194, 14.4599],
  [1572161932176, 12.66198],
  [1572248361915, 13.88375],
  [1572421123362, 13.86875],
  [1572507562228, 13.8825],
  [1572593960084, 13.9425],
  [1572680329792, 14.4807],
  [1572766617973, 14.4807],
  [1572853114639, 14.4807],
  [1572939505374, 15.0633],
  [1573025879147, 14.763861],
  [1573112315656, 14.9715],
  [157

## Cambiar de UNIX a UTF

In [2]:
timestamp  = 1563092869548
timestamp = timestamp / 1000 # 2019-07-14 08:27:49.548000
date = datetime.datetime.utcfromtimestamp(timestamp)

In [14]:
import xml.etree.ElementTree as ET
import pandas as pd

# Parsear el archivo XML
tree = ET.parse('cards.xml')
root = tree.getroot()

# Inicializar listas para almacenar datos
data = []

# Iterar a través de los elementos del XML y extraer datos
for elemento in root:
    fila = {}
    for subelemento in elemento:
        fila[subelemento.tag] = subelemento.text
    data.append(fila)

# Crear un DataFrame a partir de los datos
df = pd.DataFrame(data)

# Mostrar el DataFrame
print(df)


                          author             createdAt  \
0  Cockatrice 2.8.0 (2021-01-26)  2023-10-02T10:34:45Z   
1                            NaN                   NaN   
2                            NaN                   NaN   

                                           sourceUrl   sourceVersion  \
0  https://www.mtgjson.com/api/v5/AllPrintings.js...  5.2.2+20231001   
1                                                NaN             NaN   
2                                                NaN             NaN   

              set            card  
0             NaN             NaN  
1  \n                         NaN  
2             NaN  \n              
